In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu

yu.flag_fast=False
yu.slName='plots_notes_NST_allRatios'

tensorQ=True

In [ ]:
ensLs=['24','48']
# ensLs=['24']
L2ens={'24':'cA211.530.24','48':'cA2.09.48'}

ensembles=[L2ens[ensL] for ensL in ensLs]

ens2data={}
for ens in ensembles:
    if tensorQ:
        path={
            'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA211.530.24_tensor.h5',
            'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_tensor.h5',
            }[ens]
        yu.gjList=['id','gx','gy','gz','gt','g5','g5gx','g5gy','g5gz','g5gt','sgmyz','sgmzx','sgmxy','sgmtx','sgmty','sgmtz']
    else:
        path={
            'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_a_cA211.530.24.h5',
            'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_c_cA2.09.48.h5',
            }[ens]
        yu.gjList=['id','gx','gy','gz','gt','g5','g5gx','g5gy','g5gz','g5gt']
    with h5py.File(path) as f:
        ens2data[ens]={'VEV':{'pi0f':f['VEV']['pi0f']['data']['pi0'][:]}}
    print('Ncfg='+str(yu.deepKey(ens2data[ens],2).shape[0]))
    print()

lat=None
data=None
Ncfg=None
Tpack=None
    
def updateEns(ens):
    global lat,data,Ncfg,Tpack
    lat=yu.LatticeEnsemble(ens)
    data=ens2data[ens]
    Ncfg=yu.deepKey(data,2).shape[0]
    Tpack=yu.TPack[ens]
updateEns(ensembles[0])

In [ ]:
with open('/p/project/pines/li47/code/temp/test_for_Yan') as f:
    t=f.read()
    t=t.splitlines()
    t=np.array([[float(ele2) for ele2 in ele.split(' ')[-2:]] for ele in t])
    dat_Ferenc=t
    # print(t)

In [ ]:
ensL='48'
updateEns(L2ens[ensL])
path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_'+L2ens[ensL]+'_pion2pt.h5'
# path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_pion2pt.h5'
datpi={}
with h5py.File(path) as f:
    for key in f['data'].keys():
        datpi[key]=f['data'][key][:]
# print(datpi.keys())

#####################################

from scipy.optimize import fsolve

fig, axs = yu.getFigAxs(1,1,scale=4)

xUnit=lat.a
yUnit=lat.aInv
# yUnit=1

mom=0

timeT=len(datpi['piC'][0])
tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))


cases=['piC','pi0_conn','pi0_disc','pi0']
# cases=['pi0']
case2color={'piC':'r','pi0_conn':'b','pi0_disc':'purple','pi0':'g'}
case2keys={'piC':['piC'],'pi0_conn':['pi0_conn'],'pi0_disc':['pi0_disc'],'pi0':['pi0_conn','pi0_disc']}

for case in cases:
    color=case2color[case]
    keys=case2keys[case]
    
    vevQ = (mom==0 and 'pi0_disc' in keys)
    t=np.sum([datpi[key] for key in keys],axis=0)
    t=t[:,:,mom]
    t=[t,data['VEV']['pi0f']]

    def func(dat):
        t,tVEV=dat
        t=np.mean(t,axis=0)
        if vevQ:
            t-=np.mean(tVEV)**2
            # t-=-10
            # t-=t[48]
        t=np.real(t)
        t=(t+np.roll(np.flip(t),1,axis=0))/2
        
        # [C(t+tau)+C(t-tau)]/2C(t)
        tau=5
        tP=np.roll(t,-tau,axis=0)
        tM=np.roll(t,+tau,axis=0)
        t=np.arccosh( (tP+tM)/(2*t) )/tau
        # t=np.array([fsolve(lambda m:np.arccosh( (tfunc_C(m,xt+tau)+tfunc_C(m,xt-tau))/(2*tfunc_C(m,xt)))/tau-ele, 0.06)[0] for xt,ele in enumerate(t)])
        
        # C(t)/C(t+tau)
        # tau=1
        # tP=np.roll(t,-tau,axis=0)
        # t=t/tP
        # t=np.array([fsolve(lambda m:(tfunc_C(m,xt)/tfunc_C(m,xt+tau))-ele, 0.06)[0] for xt,ele in enumerate(t)])
        
        # t=(t+np.roll(np.flip(t),1,axis=0))/2
        # t=np.log(t/np.roll(t,-1,axis=0))
        return t
        
    mean,err,cov=yu.jackknife(t,func)
    # mean=[func(t)]
    tmin=1;tmax=timeT
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    # tMean=mean[0][tmin:tmax]; tErr=tMean*0

    irow=icol=0
    axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean*yUnit,tErr*yUnit,color=color,label=case)

for (tau,t0,color) in [(2,2,'orange')]:
    for case in ['pi0']:
        keys=case2keys[case]
        
        vevQ = (mom==0 and 'pi0_disc' in keys)
        t=np.sum([datpi[key] for key in keys],axis=0)
        t=t[:,:,mom]
        t=[t,data['VEV']['pi0f']]

        def func(dat):
            t,tVEV=dat
            t=np.complex256(t)
            t=np.mean(t,axis=0)
            t=np.complex128(t)
            if vevQ:
                t-=np.mean(tVEV)**2
                # t-=-10
                # t-=t[48]
            
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            
            t=(np.roll(t,-tau,axis=0)-np.roll(t,-t0-tau,axis=0))/(t-np.roll(t,-t0,axis=0))
            t=np.array([fsolve(lambda m:((tfunc_C(m,xt+tau)-tfunc_C(m,xt+tau+t0))/(tfunc_C(m,xt)-tfunc_C(m,xt+t0)))-ele, 0.0593)[0] for xt,ele in enumerate(t)])
            return t
            
        mean,err,cov=yu.jackknife(t,func)
        # mean=[func(t)]
        tmin=1;tmax=timeT
        tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
        # tMean=mean[0][tmin:tmax]; tErr=tMean*0

        irow=icol=0
        axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean*yUnit,tErr*yUnit,color=color,label=(tau,t0))
    
    
axs[0,0].legend(fontsize=20)

xlim=[0,8]
t_mean,t_err=np.array([0.06196,0.00009])*lat.aInv
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='r',alpha=0.2)
t_mean,t_err=np.array([0.1191,0.0005])*lat.aInv
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
t_mean,t_err=np.array([0.0593,0.0027])*lat.aInv
print(t_mean,t_err)
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='g',alpha=0.2)

# tMean=dat_Ferenc[:,0]; tErr=dat_Ferenc[:,1]
# irow=icol=0
# axs[irow,icol].errorbar(np.arange(tmin,tmin+len(tMean))*xUnit,tMean*yUnit,tErr*yUnit,color='black',label=case)


axs[irow,icol].set_ylim([0,500])
# axs[irow,icol].set_ylim([70,130])

# axs[irow,icol].set_ylim([1500,2500])

plt.tight_layout()
plt.savefig('fig/1.3.pdf')
# plt.close()

None

In [ ]:
ensL='48'
updateEns(L2ens[ensL])
path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_'+L2ens[ensL]+'_pion2pt.h5'
# path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_pion2pt.h5'
datpi={}
with h5py.File(path) as f:
    for key in f['data'].keys():
        datpi[key]=f['data'][key][:]
# print(datpi.keys())

#####################################

from scipy.optimize import fsolve

fig, axs = yu.getFigAxs(1,1,scale=4)

xUnit=lat.a
yUnit=lat.aInv
# yUnit=1

mom=0

timeT=len(datpi['piC'][0])
tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))


cases=['piC','pi0_conn','pi0_disc','pi0']
# cases=['pi0']
case2color={'piC':'r','pi0_conn':'b','pi0_disc':'purple','pi0':'g'}
case2keys={'piC':['piC'],'pi0_conn':['pi0_conn'],'pi0_disc':['pi0_disc'],'pi0':['pi0_conn','pi0_disc']}

dt=0.1
for (tau,t0,color,shift) in [(1,1,'red',1-dt*2),(1,3,'green',2-dt),(3,1,'blue',2),(2,2,'purple',2+dt),(3,3,'cyan',3+2*dt)]:
    for case in ['pi0','piC']:
        keys=case2keys[case]
        
        vevQ = (mom==0 and 'pi0_disc' in keys)
        t=np.sum([datpi[key] for key in keys],axis=0)
        t=t[:,:,mom]
        t=[t,data['VEV']['pi0f']]

        def func(dat):
            t,tVEV=dat
            t=np.complex256(t)
            t=np.mean(t,axis=0)
            t=np.complex128(t)
            if vevQ:
                t-=np.mean(tVEV)**2
                # t-=-10
                # t-=t[48]
            
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            
            t=(np.roll(t,-tau,axis=0)-np.roll(t,-t0-tau,axis=0))/(t-np.roll(t,-t0,axis=0))
            t=np.array([fsolve(lambda m:((tfunc_C(m,xt+tau)-tfunc_C(m,xt+tau+t0))/(tfunc_C(m,xt)-tfunc_C(m,xt+t0)))-ele, 0.0593)[0] for xt,ele in enumerate(t)])
            t=[ele if ele!=0.0593 else np.NaN for ele in t]
            return t
            
        mean,err,cov=yu.jackknife(t,func)
        # mean=[func(t)]
        tmin=1;tmax=24
        tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
        # tMean=mean[0][tmin:tmax]; tErr=tMean*0

        irow=icol=0
        axs[irow,icol].errorbar((np.arange(tmin,tmax)+shift)*xUnit,tMean*yUnit,tErr*yUnit,color=color,label=(tau,t0))
    
    
axs[0,0].legend(fontsize=20)

xlim=[0,2.5]
t_mean,t_err=np.array([0.06196,0.00009])*lat.aInv
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='r',alpha=0.2)
t_mean,t_err=np.array([0.1191,0.0005])*lat.aInv
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
t_mean,t_err=np.array([0.0593,0.0027])*lat.aInv
print(t_mean,t_err)
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='g',alpha=0.2)

# tMean=dat_Ferenc[:,0]; tErr=dat_Ferenc[:,1]
# irow=icol=0
# axs[irow,icol].errorbar(np.arange(tmin,tmin+len(tMean))*xUnit,tMean*yUnit,tErr*yUnit,color='black',label=case)


axs[irow,icol].set_ylim([0,200])
# axs[irow,icol].set_ylim([70,130])

# axs[irow,icol].set_ylim([1500,2500])

plt.tight_layout()
plt.savefig('fig/1.4.pdf')
# plt.close()

None

In [ ]:
# arccosh analysis 1

ensL='48'
updateEns(L2ens[ensL])
path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_'+L2ens[ensL]+'_pion2pt.h5'
# path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_pion2pt.h5'
datpi={}
with h5py.File(path) as f:
    for key in f['data'].keys():
        datpi[key]=f['data'][key][:]
# print(datpi.keys())

#####################################

from scipy.optimize import fsolve

fig, axs = yu.getFigAxs(1,1,scale=4)

xUnit=lat.a
yUnit=lat.aInv
# yUnit=1

mom=0

timeT=len(datpi['piC'][0])
tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))


cases=['piC','pi0_conn','pi0_disc','pi0']
# cases=['pi0']
case2color={'piC':'r','pi0_conn':'b','pi0_disc':'purple','pi0':'g'}
case2keys={'piC':['piC'],'pi0_conn':['pi0_conn'],'pi0_disc':['pi0_disc'],'pi0':['pi0_conn','pi0_disc']}

dt=0.1
for (tau,color,shift) in [(1,'red',1-dt*2),(2,'green',2-dt),(3,'blue',2),(4,'purple',2+dt),(5,'cyan',3+2*dt)]:
    for case in ['pi0']:
        keys=case2keys[case]
        
        vevQ = (mom==0 and 'pi0_disc' in keys)
        t=np.sum([datpi[key] for key in keys],axis=0)
        t=t[:,:,mom]
        t=[t,data['VEV']['pi0f']]

        def func(dat):
            t,tVEV=dat
            t=np.complex256(t)
            t=np.mean(t,axis=0)
            t=np.complex128(t)
            if vevQ:
                t-=np.mean(tVEV)**2
                # t-=-10
                # t-=t[48]
            
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            
            # [C(t+tau)+C(t-tau)]/2C(t)
            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            # t=np.arccosh( (tP+tM)/(2*t) )/tau
            t=(tP+tM)/(2*t)/yUnit
            
            return t
            
        mean,err,cov=yu.jackknife(t,func)
        # mean=[func(t)]
        tmin=1;tmax=24
        tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
        # tMean=mean[0][tmin:tmax]; tErr=tMean*0

        irow=icol=0
        axs[irow,icol].errorbar((np.arange(tmin,tmax)+shift)*xUnit,tMean*yUnit,tErr*yUnit,color=color,label=(tau))
    
    
axs[0,0].legend(fontsize=20)

# xlim=[0,2.5]
# t_mean,t_err=np.array([0.06196,0.00009])*lat.aInv
# axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='r',alpha=0.2)
# t_mean,t_err=np.array([0.1191,0.0005])*lat.aInv
# axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
# t_mean,t_err=np.array([0.0593,0.0027])*lat.aInv
# print(t_mean,t_err)
# axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='g',alpha=0.2)

# tMean=dat_Ferenc[:,0]; tErr=dat_Ferenc[:,1]
# irow=icol=0
# axs[irow,icol].errorbar(np.arange(tmin,tmin+len(tMean))*xUnit,tMean*yUnit,tErr*yUnit,color='black',label=case)


# axs[irow,icol].set_ylim([0,200])
# axs[irow,icol].set_ylim([70,130])

# axs[irow,icol].set_ylim([1500,2500])

plt.tight_layout()
plt.savefig('fig/cosh_pre.pdf')
# plt.close()

None

In [ ]:
# arccosh analysis 2

ensL='48'
updateEns(L2ens[ensL])
path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_'+L2ens[ensL]+'_pion2pt.h5'
# path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_pion2pt.h5'
datpi={}
with h5py.File(path) as f:
    for key in f['data'].keys():
        datpi[key]=f['data'][key][:]
# print(datpi.keys())

#####################################

from scipy.optimize import fsolve

fig, axs = yu.getFigAxs(1,1,scale=4)


xUnit=lat.a
yUnit=lat.aInv
# yUnit=1

mom=0

timeT=len(datpi['piC'][0])
tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))


cases=['piC','pi0_conn','pi0_disc','pi0']
# cases=['pi0']
case2color={'piC':'r','pi0_conn':'b','pi0_disc':'purple','pi0':'g'}
case2keys={'piC':['piC'],'pi0_conn':['pi0_conn'],'pi0_disc':['pi0_disc'],'pi0':['pi0_conn','pi0_disc']}

dt=0.1
for (tau,color,shift) in [(1,'red',1-dt*2),(2,'green',2-dt),(3,'blue',2),(4,'purple',2+dt),(5,'cyan',3+2*dt)]:
    x=np.arange(0.99,1.1,0.0002)
    y=np.arccosh(x)/tau
    axs[0,0].plot(x,y,'-',color=color)
    
    for case in ['pi0']:
        keys=case2keys[case]
        
        vevQ = (mom==0 and 'pi0_disc' in keys)
        t=np.sum([datpi[key] for key in keys],axis=0)
        t=t[:,:,mom]
        t=[t,data['VEV']['pi0f']]

        def func(dat):
            t,tVEV=dat
            t=np.complex256(t)
            t=np.mean(t,axis=0)
            t=np.complex128(t)
            if vevQ:
                t-=np.mean(tVEV)**2
            
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            
            # [C(t+tau)+C(t-tau)]/2C(t)
            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            
            t=(tP+tM)/(2*t)
            
            return t
            
        mean,err,cov=yu.jackknife(t,func)
        
        def func(dat):
            t,tVEV=dat
            t=np.complex256(t)
            t=np.mean(t,axis=0)
            t=np.complex128(t)
            if vevQ:
                t-=np.mean(tVEV)**2
            
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            
            # [C(t+tau)+C(t-tau)]/2C(t)
            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            
            t=np.arccosh( (tP+tM)/(2*t) )/tau
            
            return t
            
        mean2,err2,cov2=yu.jackknife(t,func)
        
        
        t=15
        x=mean[0][t]; xerr=err[0][t]
        y=mean2[0][t]; yerr=err2[0][t]
        

        irow=icol=0
        axs[irow,icol].errorbar(x,y,yerr,xerr=xerr,color=color,label=(tau))
    
    
axs[0,0].legend(fontsize=20)

axs[0,0].set_xlim([1,1.05])
axs[0,0].set_ylim([0,0.1])

plt.tight_layout()
plt.savefig('fig/arccoshBYtau.pdf')
# plt.close()

None

In [ ]:
ensL='48'
updateEns(L2ens[ensL])
path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_'+L2ens[ensL]+'_pion2pt.h5'
# path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_pion2pt.h5'
datpi={}
with h5py.File(path) as f:
    for key in f['data'].keys():
        datpi[key]=f['data'][key][:]
# print(datpi.keys())

#####################################

from scipy.optimize import fsolve

fig, axs = yu.getFigAxs(1,1,scale=4)

xUnit=lat.a
yUnit=lat.aInv
# yUnit=1

mom=0

timeT=len(datpi['piC'][0])
tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))


cases=['piC','pi0_conn','pi0_disc','pi0']
# cases=['pi0']
case2color={'piC':'r','pi0_conn':'b','pi0_disc':'purple','pi0':'g'}
case2keys={'piC':['piC'],'pi0_conn':['pi0_conn'],'pi0_disc':['pi0_disc'],'pi0':['pi0_conn','pi0_disc']}

dt=0.1
for (tau,t0,color,shift) in [(1,1,'red',1-dt*2),(1,3,'green',2-dt),(3,1,'blue',2),(2,2,'purple',2+dt),(3,3,'cyan',3+2*dt)]:
    for case in ['diff']:
        keys_C=case2keys['piC']
        keys_0=case2keys['pi0']
        
        # vevQ = (mom==0 and 'pi0_disc' in keys)
        t=np.sum([datpi[key] for key in keys_C],axis=0)
        ttC=t[:,:,mom]
        t=np.sum([datpi[key] for key in keys_0],axis=0)
        tt0=t[:,:,mom]
        t=[ttC,tt0]

        def func(dat):
            ttC,tt0=dat
            
            #############################
            
            t=np.complex256(ttC)
            t=np.mean(t,axis=0)
            t=np.complex128(t)
            # if vevQ:
            #     t-=np.mean(tVEV)**2
                # t-=-10
                # t-=t[48]
            
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            
            t=(np.roll(t,-tau,axis=0)-np.roll(t,-t0-tau,axis=0))/(t-np.roll(t,-t0,axis=0))
            ttC=np.array([fsolve(lambda m:((tfunc_C(m,xt+tau)-tfunc_C(m,xt+tau+t0))/(tfunc_C(m,xt)-tfunc_C(m,xt+t0)))-ele, 0.0593)[0] for xt,ele in enumerate(t)])
            
            ###########################
            
            t=np.complex256(tt0)
            t=np.mean(t,axis=0)
            t=np.complex128(t)
            # if vevQ:
            #     t-=np.mean(tVEV)**2
                # t-=-10
                # t-=t[48]
            
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            
            t=(np.roll(t,-tau,axis=0)-np.roll(t,-t0-tau,axis=0))/(t-np.roll(t,-t0,axis=0))
            tt0=np.array([fsolve(lambda m:((tfunc_C(m,xt+tau)-tfunc_C(m,xt+tau+t0))/(tfunc_C(m,xt)-tfunc_C(m,xt+t0)))-ele, 0.0593)[0] for xt,ele in enumerate(t)])
            
            ###########################
            
            t=ttC-tt0
            
            return t
            
        mean,err,cov=yu.jackknife(t,func)
        # mean=[func(t)]
        tmin=1;tmax=24
        tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
        # tMean=mean[0][tmin:tmax]; tErr=tMean*0

        irow=icol=0
        axs[irow,icol].errorbar((np.arange(tmin,tmax)+shift)*xUnit,tMean*yUnit,tErr*yUnit,color=color,label=(tau,t0))
    
    
axs[0,0].legend(fontsize=20)

xlim=[0,2.5]
t_mean,t_err=np.array([0.06196,0.00009])*lat.aInv
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='r',alpha=0.2)
t_mean,t_err=np.array([0.1191,0.0005])*lat.aInv
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='b',alpha=0.2)
t_mean,t_err=np.array([0.0593,0.0027])*lat.aInv
print(t_mean,t_err)
axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='g',alpha=0.2)

# tMean=dat_Ferenc[:,0]; tErr=dat_Ferenc[:,1]
# irow=icol=0
# axs[irow,icol].errorbar(np.arange(tmin,tmin+len(tMean))*xUnit,tMean*yUnit,tErr*yUnit,color='black',label=case)


axs[irow,icol].set_ylim([-20,50])
# axs[irow,icol].set_ylim([70,130])

# axs[irow,icol].set_ylim([1500,2500])

plt.tight_layout()
plt.savefig('fig/1.4.pdf')
# plt.close()

None